### This code select the stimuli and generate the practice spreadsheet for gorilla experiment (need to change the prompt to adopt different task)

### Library & Define function

In [12]:
import os
import glob
import random
import shutil
import openpyxl

In [13]:
def combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword):
    os.makedirs(new_folder, exist_ok=True)

    new_wb = openpyxl.Workbook()
    new_ws = new_wb.active

    header_written = False

    for subfolder in subfolders:
        xlsx_files = [f for f in os.listdir(os.path.join(source_folder, subfolder)) if f.endswith('.xlsx')]

        # select xlsx_file with keyword in its name
        xlsx_file = next((f for f in xlsx_files if keyword in f.lower()), None)
        if not xlsx_file:
            continue

        wb = openpyxl.load_workbook(os.path.join(source_folder, subfolder, xlsx_file))
        ws = wb.active

        if not header_written:
            new_ws.append([cell.value for cell in ws[1]])
            header_written = True

        rows_to_select = random.sample(range(2, ws.max_row + 1), min(ws.max_row - 1, selected_rows))

        for selected_row in rows_to_select:
            new_ws.append([cell.value for cell in ws[selected_row]])

            # Find the "Audio_Filename" column index
            audio_filename_col_index = None
            for idx, cell in enumerate(ws[1]):
                if cell.value == "Audio_Filename":
                    audio_filename_col_index = idx + 1
                    break

            if audio_filename_col_index:
                audio_filename = ws.cell(row=selected_row, column=audio_filename_col_index).value
                audio_file_path = os.path.join(source_folder, subfolder, audio_filename)
                if os.path.isfile(audio_file_path):
                    new_audio_filename = os.path.splitext(audio_filename)[0] + '_prac' + os.path.splitext(audio_filename)[1]
                    new_audio_file_path = os.path.join(new_folder, new_audio_filename)
                    shutil.copy(audio_file_path, new_audio_file_path)

                    # Update the "Audio_Filename" in the new xlsx file
                    new_ws.cell(row=new_ws.max_row, column=audio_filename_col_index).value = new_audio_filename

    new_wb.save(os.path.join(new_folder, 'combined_rows_prac.xlsx'))


In [3]:
def create_combined_xlsx(file1, file2, new_folder, input_string, file_name, task_type):
    AP_intro = '<p style="font-size: 32px;"> In this practice section, you will hear a sequence of complex tones/everyday sounds, a pause, then a single complex tone/everyday sound. Your task is to determine whether the single complex tone/everyday sound was present or absent from the sequence. </p>'
    SD_intro = '<p style="font-size: 32px;"> In this practice section, you will hear a sequence of complex tones/everyday sounds, a pause, then a second sequence of complex tone/everyday sound. Your task is to determine whether the 2nd sequence of complex tone/everyday sound was same or different from the 1st sequence. </p>'
    # Load workbooks and worksheets
    wb1 = openpyxl.load_workbook(file1)
    ws1 = wb1.active

    wb2 = openpyxl.load_workbook(file2)
    ws2 = wb2.active

    new_wb = openpyxl.Workbook()
    new_ws = new_wb.active

    # 1. Copy the header from the 1st xlsx
    header = [cell.value for cell in ws1[1]]
    new_ws.append(header)

    # Find 'display', 'text', 'randomise_trials', and 'embedded' column indices
    display_col_index = header.index('display') + 1
    text_col_index = header.index('text') + 1
    randomise_trials_col_index = header.index('randomise_trials') + 1
    embedded_col_index = header.index('embedded') + 1


    # 2. Add a line with 'text' under the 'display' column and 'a' under the 'text' column
    new_row = [''] * len(header)
    new_row[display_col_index - 1] = 'text'
    #new_row[text_col_index - 1] = '<p style="font-size: 32px;"> In this practice section, you will hear a sequence of syllables with different pitches, a pause, then a single syllable. Your task is to determine whether the PITCH itself was present or absent from the sequence. </p>'
    new_row[text_col_index - 1] = AP_intro if task_type in ['task-ap-feedback'] else SD_intro
    new_ws.append(new_row)

    # 3. Copy all the nonheader lines from the first xlsx file and change the 'embedded' column with input string
    for row in ws1.iter_rows(min_row=2):
        new_row = [cell.value for cell in row]
        new_row[embedded_col_index - 1] = input_string
        new_row[display_col_index - 1] = task_type
        new_ws.append(new_row)

    # 4. Add a line with 'text' under the 'display' column and 'b' under the 'text' column
    new_row = [''] * len(header)
    new_row[display_col_index - 1] = 'text'
    #new_row[text_col_index - 1] = '<p style="font-size: 32px;"> In this final practice section, you will hear a sequence of syllables with different pitches, a pause, then a single syllable. Your task is to determine whether the SYLLABLE itself was present or absent from the sequence. </p>'
    new_row[text_col_index - 1] = AP_intro if task_type in ['task-ap-feedback'] else SD_intro
    new_ws.append(new_row)

    # 5. Copy all the nonheader lines from the second xlsx file, change the 'randomise_trials' values to 2, and change the 'embedded' column with input string
    for row in ws2.iter_rows(min_row=2):
        new_row = [cell.value for cell in row]
        new_row[randomise_trials_col_index - 1] = 2
        new_row[embedded_col_index - 1] = input_string
        new_row[display_col_index - 1] = task_type
        new_ws.append(new_row)

    new_wb.save(os.path.join(new_folder, file_name))

### AP

#### Pitch and syllable

In [4]:
# ### Parameters
# selected_rows = 4  # specify the number of rows to select
# keyword = 'combined'  # specify the keyword

# ###~~~~~~~~~~~~~~ AP-Pitch ~~~~~~~~~~~~~~~~~
# source_folder = "E:/cmu/lab project/stimuli generation/V5-syllable-4-semitone-interval-10-rand/task_type_ap/Block_2_pitch"
# new_folder = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_AP/AP_2_section_Practice/Pitch"

# #specify subfolder names
# subfolders = ['length5']  # specify the subfolders you want to process
# #subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

# if not os.path.exists(new_folder):
#         os.makedirs(new_folder)

# combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)


# ###~~~~~~~~~~~~~~ AP-Word ~~~~~~~~~~~~~~~
# source_folder = "E:/cmu/lab project/stimuli generation/V5-syllable-4-semitone-interval-10-rand/task_type_ap/Block_1_word"
# new_folder = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_AP/AP_2_section_Practice/Word"

# #specify subfolder names
# subfolders = ['length5']  # specify the subfolders you want to process
# #subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

# if not os.path.exists(new_folder):
#         os.makedirs(new_folder)

# combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

#### complex tone vs everyday sound

In [5]:
### Parameters
selected_rows = 4  # specify the number of rows to select
keyword = 'metadata'  # specify the keyword
outpath = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/output_spread_sheet"

###~~~~~~~~~~~~~~ AP-complex tone ~~~~~~~~~~~~~~~~
source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/stimuli/PA/block2_complex_tones"
new_folder = outpath + "/AP_practice_section/complex_notes"

#specify subfolder names
subfolders = ['length4']  # specify the subfolders you want to process
#subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

if not os.path.exists(new_folder):
    os.makedirs(new_folder)

combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

###~~~~~~~~~~~~~ AP-everyday sound ~~~~~~~~~~~~~~~~~~~~
source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/stimuli/PA/block1_everyday_sounds"
new_folder = outpath + "/AP_practice_section/everyday_sounds"

#specify subfolder names
subfolders = ['length4']  # specify the subfolders you want to process
#subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

if not os.path.exists(new_folder):
    os.makedirs(new_folder)

combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

#### complex tone vs cat 

In [6]:
# ### Parameters
# selected_rows = 4  # specify the number of rows to select
# keyword = 'metadata'  # specify the keyword
# outpath = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/output_spread_sheet"

# ###~~~~~~~~~~~~~~ AP-complex tone ~~~~~~~~~~~~~~~~ 
# source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/stimuli/PA/block1_complex_tones" # Don't choose same block, or randomized block value mixed
# new_folder = outpath + "/AP_practice_section/complex_notes"

# #specify subfolder names
# subfolders = ['length4']  # specify the subfolders you want to process
# #subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

# if not os.path.exists(new_folder):
#     os.makedirs(new_folder)

# combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

# ###~~~~~~~~~~~~~ AP-everyday sound ~~~~~~~~~~~~~~~~~~~~
# source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/stimuli/PA/block2_cat_mew_sounds" # Don't choose same block, or randomized block value mixed
# new_folder = outpath + "/AP_practice_section/cat_sounds"

# #specify subfolder names
# subfolders = ['length4']  # specify the subfolders you want to process
# #subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

# if not os.path.exists(new_folder):
#     os.makedirs(new_folder)

# combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

#### complex tones vs dog sounds tbd

#### merge the practice xlsx and wav files

In [7]:
# ### pitch variated syllable
# file1 = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_AP/AP_2_section_Practice/Pitch/combined_rows_prac.xlsx"
# file2 = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_AP/AP_2_section_Practice/Word/combined_rows_prac.xlsx"
# output_file = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_AP/AP_2_section_Practice"

### complex note and everyday sound 
# change the instruction language in the code
file1 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/output_spread_sheet/AP_practice_section/complex_notes/combined_rows_prac.xlsx"
file2 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/output_spread_sheet/AP_practice_section/everyday_sounds/combined_rows_prac.xlsx"
output_file = outpath + "/AP_practice_section"

# ### complex note and cat sound
# file1 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/output_spread_sheet/AP_practice_section/complex_notes/combined_rows_prac.xlsx"
# file2 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/output_spread_sheet/AP_practice_section/cat_sounds/combined_rows_prac.xlsx"
# output_file = outpath + "/AP_practice_section"


if not os.path.exists(output_file):
        os.makedirs(output_file)

### create the pratice 1
create_combined_xlsx(file1, file2, output_file,'prac_corr','AP_practice1.xlsx', 'task-ap-feedback')
### create the pratice 2
create_combined_xlsx(file1, file2, output_file,'prac_corr2','AP_practice2.xlsx', 'task-ap-feedback')

### SD

#### pitch variated syllable

In [8]:
# ### Parameters
# selected_rows = 4  # specify the number of rows to select
# keyword = 'combined'  # specify the keyword

# ###~~~~~~~~~~~~~~ SD-Pitch ~~~~~~~~~~~~~~~~~
# source_folder = "E:/cmu/lab project/stimuli generation/V5-syllable-4-semitone-interval-10-rand/task_type_sd/Block_2_pitch"
# new_folder = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_SD/SD_2_section_Practice/Pitch"

# #specify subfolder names
# subfolders = ['length5']  # specify the subfolders you want to process
# #subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

# if not os.path.exists(new_folder):
#         os.makedirs(new_folder)

# combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)


# ###~~~~~~~~~~~~~~ SD-Word ~~~~~~~~~~~~~~~
# source_folder = "E:/cmu/lab project/stimuli generation/V5-syllable-4-semitone-interval-10-rand/task_type_sd/Block_1_word"
# new_folder = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_SD/SD_2_section_Practice/Word"

# #specify subfolder names
# subfolders = ['length5']  # specify the subfolders you want to process
# #subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

# if not os.path.exists(new_folder):
#         os.makedirs(new_folder)

# combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

#### complex tone vs everyday sound

In [9]:
### Parameters
selected_rows = 4  # specify the number of rows to select
keyword = 'metadata'  # specify the keyword
outpath = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/output_spread_sheet"

###~~~~~~~~~~~~~~ SD-complex tone ~~~~~~~~~~~~~~~~
source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/stimuli/SD/block2_complex_tones"
new_folder = outpath + "/SD_practice_section/complex_notes"

#specify subfolder names
subfolders = ['length4']  # specify the subfolders you want to process
#subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

if not os.path.exists(new_folder):
    os.makedirs(new_folder)

combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

###~~~~~~~~~~~~~ SD-everyday sound ~~~~~~~~~~~~~~~~~~~~
#find a random block as practice section source folder
source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/stimuli/SD/block3_everyday_sounds"
new_folder = outpath + "/SD_practice_section/everyday_sounds"

#specify subfolder names
subfolders = ['length4']  # specify the subfolders you want to process
#subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

if not os.path.exists(new_folder):
    os.makedirs(new_folder)

combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

#### complex tone vs cat sounds

In [10]:
# ### Parameters
# selected_rows = 4  # specify the number of rows to select
# keyword = 'metadata'  # specify the keyword
# outpath = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/output_spread_sheet"

# ###~~~~~~~~~~~~~~ SD-complex tone ~~~~~~~~~~~~~~~~
# source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/stimuli/SD/block1_complex_tones" # Don't choose same block, or randomized block value mixed
# new_folder = outpath + "/SD_practice_section/complex_notes"

# #specify subfolder names
# subfolders = ['length4']  # specify the subfolders you want to process
# #subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

# if not os.path.exists(new_folder):
#     os.makedirs(new_folder)

# combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

# ###~~~~~~~~~~~~~ SD-everyday sound ~~~~~~~~~~~~~~~~~~~~
# #find a random block as practice section source folder
# source_folder = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/stimuli/SD/block2_cat_mew_sounds" # Don't choose same block, or randomized block value mixed
# new_folder = outpath + "/SD_practice_section/cat_sounds"

# #specify subfolder names
# subfolders = ['length4']  # specify the subfolders you want to process
# #subfolders = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))] # all subfolders

# if not os.path.exists(new_folder):
#     os.makedirs(new_folder)

# combine_rows_from_xlsx_files(source_folder, new_folder, subfolders, selected_rows, keyword)

#### complex tone vs dog sounds TBD

#### merge the practice xlsx and wav files

In [11]:
# ### pitch variated syllable
# file1 = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_SD/SD_2_section_Practice/Pitch/combined_rows_prac.xlsx"
# file2 = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_SD/SD_2_section_Practice/Word/combined_rows_prac.xlsx"
# output_file = "E:/cmu/lab project/stimuli generation/V5-spread_sheet/Task_SD/SD_2_section_Practice"

### complex note and everyday sound 
# change the instruction in the code
file1 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/output_spread_sheet/SD_practice_section/complex_notes/combined_rows_prac.xlsx"
file2 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_everydaySound/output_spread_sheet/SD_practice_section/everyday_sounds/combined_rows_prac.xlsx"
output_file = outpath + "/SD_practice_section"

# ### complex note and cat sound 
# # change the instruction in the code
# file1 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/output_spread_sheet/SD_practice_section/complex_notes/combined_rows_prac.xlsx"
# file2 = "/Volumes/T7/CMU LiMN Research/perceptual organization/stimuli generation/len4-complexTone_cat/output_spread_sheet/SD_practice_section/cat_sounds/combined_rows_prac.xlsx"
# output_file = outpath + "/SD_practice_section"

if not os.path.exists(output_file):
        os.makedirs(output_file)

### create the pratice 1
create_combined_xlsx(file1, file2, output_file,'prac_corr3','SD_practice1.xlsx', 'task-sd-feedback')
### create the pratice 2
create_combined_xlsx(file1, file2, output_file,'prac_corr4','SD_practice2.xlsx', 'task-sd-feedback')